<a href="https://colab.research.google.com/github/Panda1304/Emotion_Embedding_Ratings/blob/main/Ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test_1.csv')

In [ ]:
print(train_data.columns)

Index(['ID', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
       'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29',
       'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34',
       'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39',
       'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44',
       'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49',
       'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54',
       'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59',
       'feature_60', 'feature_61', 'feature_62', 'feature_63', 'score'],
      d

In [ ]:
X_train = train_data.drop(columns=['ID','score']).values
y_train = train_data['score'].values  # The actual ratings (score)

In [ ]:
X_test = test_data.drop(columns=['ID']).values

In [ ]:
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [ ]:
cnn_input = layers.Input(shape=(64,))  # 64-dimensional embeddings
x = layers.Reshape((64, 1))(cnn_input)  #  (1D Conv)
x = layers.Conv1D(32, 3, activation='relu')(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Conv1D(64, 3, activation='relu')(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(1, activation='linear')(x)

In [ ]:
# CNN Model
cnn_model = models.Model(inputs=cnn_input, outputs=x)
cnn_model.compile(optimizer='adam', loss='mse')

In [ ]:
cnn_model.fit(X_train_normalized, y_train, epochs=10, batch_size=42, verbose=1)

Epoch 1/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 1.6858
Epoch 2/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.7792
Epoch 3/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.7294
Epoch 4/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.7128
Epoch 5/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.7290
Epoch 6/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.7173
Epoch 7/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.7047
Epoch 8/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.6927
Epoch 9/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.6944
Epoch 10/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.6950


In [ ]:
train_cnn_features = cnn_model.predict(X_train_normalized)
test_cnn_features = cnn_model.predict(X_test_normalized)

984/984 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor(n_neighbors=30190,p=1,weights='distance',leaf_size=20)
knn.fit(train_cnn_features, y_train)

KNeighborsRegressor(leaf_size=20, n_neighbors=30190, p=1, weights='distance')

In [ ]:
predictions = knn.predict(test_cnn_features)

In [ ]:
submission = pd.DataFrame({
    'ID': test_data['ID'],
    'score': predictions
})

In [ ]:
submission.to_csv('submission_148.csv', index=False)

In [ ]:
print("Submission file created: 'submission.csv'")

Submission file created: 'submission.csv'
